# TUTORIAL: LIMPIEZA DE DATOS CON PYTHON Y PANDAS

# 1. El problema del negocio

Una entidad bancaria contrata a una empresa de marketing encargada de contactar telefónicamente a posibles clientes para determinar si están interesados o no en adquirir un certificado de depósito a término con el banco.

¿Qué perfil tienen los clientes con mayor potencial de conversión?

#2. El set de datos

La información recolectada por la empresa de mercadeo se encuentra en un archivo CSV (`dataset_banco.csv`) con 45215 filas y 17 columnas.

Cada registro contiene 16 características (las primeras 16 columnas) y una categoría ("yes" o "no" dependiendo de si la persona está o no interesada en adquirir el producto). Las columnas son:

1. "age":  edad (numérica)
2. "job": tipo de trabajo (categórica: "admin.", "unknown", "unemployed", "management", "housemaid", "entrepreneur", "student", "blue-collar","self-employed", "retired", "technician", "services")
3. "marital": estado civil (categórica: "married", "divorced", "single")
4. "education": nivel educativo (categórica: "unknown", "secondary", "primary", "tertiary")
5. "default": si dejó de pagar sus obligaciones (categórica: "yes", "no")
6. "balance": saldo promedio anual en euros (numérica)
7. "housing": ¿tiene o no crédito hipotecario? (categórica: "yes", "no")
8. "loan": ¿tiene créditos de consumo? (categórica: "yes", "no")
9. "contact": medio a través del cual fue contactado (categórica: "unknown", "telephone", "cellular")
10. "day": último día del mes en el que fue contactada (numérica)
11. "month": último mes en el que fue contactada (categórica: "jan", "feb", "mar", ..., "nov", "dec")
12. "duration": duración (en segundos) del último contacto (numérica)
13. "campaign": número total de veces que fue contactada durante la campaña (numérica)
14. "pdays": número de días transcurridos después de haber sido contactado antes de la campaña actual (numérica. -1 indica que no fue contactado previamente)
15. "previous": número de veces que ha sido contactada antes de esta campaña (numérica)
16. "poutcome": resultado de la campaña de marketing anterior (categórica: "unknown", "other", "failure", "success")
17. "y": categoría ¿el cliente se suscribió a un depósito a término? (categórica: "yes", "no")

#3. Una primera mirada al dataset

In [5]:
# Importar librerías
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:

# Carga el archivo desde los archivos del cuaderno
df = pd.read_csv('dataset_banco.csv')

# Haz algo con el DataFrame, por ejemplo, muestra las primeras filas
print(df.head())


   age           job  marital  education default  balance housing loan  \
0   58    management  married   tertiary      no   2143.0     yes   no   
1   44    technician   single  secondary      no     29.0     yes   no   
2   33  entrepreneur  married  secondary      no      2.0     yes  yes   
3   47   blue-collar  married    unknown      no   1506.0     yes   no   
4   33       unknown   single    unknown      no      1.0      no   no   

   contact  day month  duration  campaign  pdays  previous poutcome   y  
0  unknown    5   may     261.0         1   -1.0         0  unknown  no  
1  unknown    5   may     151.0         1   -1.0         0  unknown  no  
2  unknown    5   may      76.0         1   -1.0         0  unknown  no  
3  unknown    5   may      92.0         1   -1.0         0  unknown  no  
4  unknown    5   may     198.0         1   -1.0         0  unknown  no  


In [7]:
# Veamos las variables categóricas y las numéricas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45215 entries, 0 to 45214
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        45215 non-null  int64  
 1   job        45213 non-null  object 
 2   marital    45214 non-null  object 
 3   education  45214 non-null  object 
 4   default    45215 non-null  object 
 5   balance    45213 non-null  float64
 6   housing    45215 non-null  object 
 7   loan       45215 non-null  object 
 8   contact    45215 non-null  object 
 9   day        45215 non-null  int64  
 10  month      45215 non-null  object 
 11  duration   45214 non-null  float64
 12  campaign   45215 non-null  int64  
 13  pdays      45214 non-null  float64
 14  previous   45215 non-null  int64  
 15  poutcome   45215 non-null  object 
 16  y          45215 non-null  object 
dtypes: float64(3), int64(4), object(10)
memory usage: 5.9+ MB


#4. Limpieza

Realizaremos el proceso de limpieza teniendo en cuenta las situaciones más comunes:

1. Datos faltantes en algunas celdas
2. Columnas irrelevantes (que no responden al problema que queremos resolver)
3. Registros (filas) repetidos
4. Valores extremos (*outliers*) en el caso de las variables numéricas. Se deben analizar en detalle pues no necesariamente la solución es eliminarlos
5. Errores tipográficos en el caso de las variables categóricas

Al final de este proceso de limpieza deberíamos tener un set de datos **íntegro**, listo para la fase de Análisis Exploratorio.

## 4.1 Datos faltantes

Comenzamos a ver que los datos no están completos, pues no todas las columnas tienen la misma cantidad de registros.

El número total de registros debería ser 45.215. Sin embargo columnas como "job", "marital", "education", "balance", "duration" y "pdays".

Por ser tan pocos los datos  faltantes optaremos por eliminar las filas correspondientes:

In [8]:
# Identificar valores faltantes en el dataset
missing_values = df.isnull().sum()

# Mostrar las columnas con valores faltantes y su cantidad
print("Valores faltantes por columna:")
print(missing_values)


Valores faltantes por columna:
age          0
job          2
marital      1
education    1
default      0
balance      2
housing      0
loan         0
contact      0
day          0
month        0
duration     1
campaign     0
pdays        1
previous     0
poutcome     0
y            0
dtype: int64


In [9]:
# debajo hago los dos seguidos porqie sino no se añaden


##4.2 Columnas irrelevantes

Una columna irrelevante puede ser:

- **Una columna que no contiene información relevante para el problema que queremos resolver**. Por ejemplo en este caso podría ser una columna que no guarde relación con el posible perfil del cliente (deporte favorito, hobbies, comida favorita, etc.)
- **Una columna categórica pero con un sólo nivel**. Por ejemplo si en la columna "job" solo tuviésemos el nivel "unknown".
- **Una columna numérica pero con un sólo valor**. Por ejemplo si en la columna "edad" todos los vlaores fuesen iguales a 50.
- **Columnas con información redundante**. Por ejemplo si además de las columnas "month" y "day" tuviésemos la columna "month-day", resultado de combinar las dos anteriores.

Si tenemos la duda de si alguna columna puede ser relevante o no lo mejor es dejarla (y más adelante en posteriores etapas podremos darnos cuenta de si se debe preservar o no).

En este caso todas las columnas pueden resultar relevantes, pero debemos verificar que no haya columnas categóricas con un sólo nivel, o columnas numéricas con un sólo valor:

Todas las columnas categóricas tienen más de 1 subnivel. No eliminaremos ninguna.

Verifiquemos lo que ocurre con las columnas numéricas:

Todas las columnas numéricas tienen desviaciones estándar ("std") diferentes de cero, lo que indica que no tienen un único valor.

Preservaremos todas las columnas numéricas.

In [10]:
# Verificar columnas categóricas con un solo nivel
single_level_categorical = []
for column in df.columns:
    if df[column].nunique() == 1:
        single_level_categorical.append(column)

# Verificar columnas numéricas con un solo valor
single_value_numeric = []
for column in df.select_dtypes(include='number').columns:
    if df[column].nunique() == 1:
        single_value_numeric.append(column)

# Mostrar columnas irrelevantes encontradas
print("Columnas categóricas con un solo nivel:", single_level_categorical)
print("Columnas numéricas con un solo valor:", single_value_numeric)


Columnas categóricas con un solo nivel: []
Columnas numéricas con un solo valor: []


### Esto quiere decir que no hay

##4.3 Filas repetidas

In [11]:
# Contar filas duplicadas
duplicates_count = df[df.duplicated()].shape[0]

# Mostrar el número de filas duplicadas
print("Número de filas duplicadas:", duplicates_count)


Número de filas duplicadas: 4


### Eliminamos las filas duplicadas

In [12]:
# Eliminar filas con valores faltantes
df_clean = df.dropna()

# Contar filas duplicadas antes de eliminarlas
duplicates_before = df_clean[df_clean.duplicated()].shape[0]

# Eliminar filas duplicadas
df_clean = df_clean.drop_duplicates()

# Contar filas duplicadas después de eliminarlas
duplicates_after = df_clean[df_clean.duplicated()].shape[0]

# Mostrar el número de filas duplicadas antes y después de la eliminación
print("Número de filas duplicadas antes de eliminarlas:", duplicates_before)
print("Número de filas duplicadas después de eliminarlas:", duplicates_after)

# Mostrar la cantidad de filas después de la limpieza completa
print("Número de filas después de la limpieza completa:", df_clean.shape[0])

Número de filas duplicadas antes de eliminarlas: 4
Número de filas duplicadas después de eliminarlas: 0
Número de filas después de la limpieza completa: 45203


##4.4 *Outliers* en las variables numéricas

No siempre se deben eliminar los *outliers* porque dependiendo de la variable numérica analizada estos pueden contener información importante.


In [13]:
import numpy as np

# Seleccionar solo las columnas numéricas de interés
numeric_columns = ["age", "day", "duration", "campaign", "previous"]
df_numeric = df_clean[numeric_columns]

# Calcular el rango intercuartílico (IQR) para cada columna numérica
Q1 = df_numeric.quantile(0.25)
Q3 = df_numeric.quantile(0.75)
IQR = Q3 - Q1

# Definir el umbral para identificar outliers
threshold = 1.5

# Identificar outliers para cada columna numérica
outliers = (df_numeric < (Q1 - threshold * IQR)) | (df_numeric > (Q3 + threshold * IQR))

# Mostrar los índices de las filas que contienen outliers en al menos una columna numérica
outliers_indices = outliers.any(axis=1)
outliers_indices = outliers_indices[outliers_indices == True].index
print("Índices de las filas con outliers en las variables numéricas seleccionadas:")
print(outliers_indices)

# Mostrar las filas que contienen outliers en las variables numéricas seleccionadas
print("Filas con outliers en las variables numéricas seleccionadas:")
print(df_clean.loc[outliers_indices])


Índices de las filas con outliers en las variables numéricas seleccionadas:
Index([   12,    37,    43,    53,    59,    61,    83,    86,    87,   109,
       ...
       45196, 45198, 45202, 45204, 45205, 45208, 45210, 45211, 45212, 45214],
      dtype='int64', length=13919)
Filas con outliers en las variables numéricas seleccionadas:
       age           job   marital  education default  balance housing loan  \
12      53    technician   married  secondary      no      6.0     yes   no   
37      53    technician   married  secondary      no     -3.0      no   no   
43      54       retired   married  secondary      no    529.0     yes   no   
53      42        admin.    single  secondary      no    -76.0     yes   no   
59      46      services   married    primary      no    179.0     yes   no   
...    ...           ...       ...        ...     ...      ...     ...  ...   
45208   73       retired   married  secondary      no   2850.0      no   no   
45210   51    technician   mar

In [14]:
import numpy as np

# Seleccionar solo las columnas numéricas de interés
numeric_columns = ["age", "day", "duration", "campaign", "previous"]
df_numeric = df_clean[numeric_columns]

# Calcular el rango intercuartílico (IQR) para cada columna numérica
Q1 = df_numeric.quantile(0.25)
Q3 = df_numeric.quantile(0.75)
IQR = Q3 - Q1

# Definir el umbral para identificar outliers
threshold = 1.5

# Identificar outliers para cada columna numérica
outliers = (df_numeric < (Q1 - threshold * IQR)) | (df_numeric > (Q3 + threshold * IQR))

# Mostrar los índices de las filas que contienen outliers en al menos una columna numérica
outliers_indices = outliers.any(axis=1)
outliers_indices = outliers_indices[outliers_indices == True].index
print("Índices de las filas con outliers en las variables numéricas seleccionadas:")
print(outliers_indices)

# Mostrar las filas que contienen outliers en las variables numéricas seleccionadas
print("Filas con outliers en las variables numéricas seleccionadas:")
print(df_clean.loc[outliers_indices, numeric_columns])


Índices de las filas con outliers en las variables numéricas seleccionadas:
Index([   12,    37,    43,    53,    59,    61,    83,    86,    87,   109,
       ...
       45196, 45198, 45202, 45204, 45205, 45208, 45210, 45211, 45212, 45214],
      dtype='int64', length=13919)
Filas con outliers en las variables numéricas seleccionadas:
       age  day  duration  campaign  previous
12      53    5    -517.0         1         0
37      53    5    1666.0         1         0
43      54    5    1492.0         1         0
53      42    5     787.0         1         0
59      46    5    1778.0         1         0
...    ...  ...       ...       ...       ...
45208   73   17     300.0         1         8
45210   51   17     977.0         3         0
45211   71   17     456.0         2         0
45212   72   17    1127.0         5         3
45214   37   17     361.0         2        11

[13919 rows x 5 columns]


### Comprobamos si hay algún valor negativo en alguna fila

In [15]:
# Seleccionar solo las columnas numéricas de interés
numeric_columns = ["age", "day", "duration", "campaign", "previous"]
df_numeric = df_clean[numeric_columns]

# Verificar si hay valores negativos en las columnas numéricas
negative_values = df_numeric[df_numeric < 0].any()

# Mostrar si hay algún valor negativo en las variables numéricas seleccionadas
print("¿Hay algún valor negativo en las variables numéricas seleccionadas?")
print(negative_values)

¿Hay algún valor negativo en las variables numéricas seleccionadas?
age         False
day         False
duration     True
campaign    False
previous    False
dtype: bool


### duration     True; quiere decir que hay un valor/es negativos en duration, lo que es imposible, asi que eliminamos esas filas del dataset.

In [16]:
# Seleccionar las filas con valores negativos en la columna "duration"
negative_duration_rows = df_clean.loc[df_clean['duration'] < 0]

# Mostrar las filas con valores negativos en la columna "duration"
print("Filas con valores negativos en 'duration':")
print(negative_duration_rows)

# Obtener los índices de las filas con valores negativos en la columna "duration"
negative_duration_indices = negative_duration_rows.index

# Eliminar las filas con valores negativos en la columna "duration"
df_clean = df_clean.drop(negative_duration_indices)

# Verificar la forma del DataFrame después de eliminar las filas con valores negativos en "duration"
print("\nDimensiones del DataFrame después de eliminar filas con valores negativos en 'duration':", df_clean.shape)


Filas con valores negativos en 'duration':
    age         job  marital  education default  balance housing loan  \
12   53  technician  married  secondary      no      6.0     yes   no   
87   41  technician  married  secondary      no   1270.0     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome    y  
12  unknown    5   may    -517.0         1   -1.0         0  unknown   no  
87  unknown    5   may   -1389.0         1   -1.0         0  unknown  yes  

Dimensiones del DataFrame después de eliminar filas con valores negativos en 'duration': (45201, 17)


### Ahora comprobamos los valores máximos de Edad, para ver si ha ocurrido algún error recogiendo la muestra.

In [17]:
# Mostrar los 25 valores máximos de la columna "age"
max_age = df_clean['age'].nlargest(25)
print("Los 25 valores máximos de 'age' son:")
print(max_age)


Los 25 valores máximos de 'age' son:
43530    776
229      530
1076     490
28152    466
6053     399
3018     332
5799     311
2229     123
33701     95
41666     95
31235     94
44265     93
44423     93
41937     92
42730     92
31071     90
43197     90
41165     89
42577     89
44895     89
44672     88
44797     88
41926     87
42084     87
42497     87
Name: age, dtype: int64


### Cogemos por ejemplo, la cifra de 117 años, que representa la persona mas longeva registrada, por lo que cualquier fila con un valor "age" mayor es eliminado del dataset

In [18]:
# Seleccionar las filas con valores de "age" mayores que 117
age_above_117_indices = df_clean.loc[df_clean['age'] > 117].index

# Eliminar las filas con valores de "age" mayores que 117
df_clean = df_clean.drop(age_above_117_indices)

# Verificar la forma del DataFrame después de eliminar las filas con "age" > 117
print("Dimensiones del DataFrame después de eliminar filas con 'age' > 117:", df_clean.shape)


Dimensiones del DataFrame después de eliminar filas con 'age' > 117: (45193, 17)


### Comprobamos si algún valor de "day" se sale de 1-31, que es el máximo de días que puede tener un mes. No hay por lo tanto esta variable es correcta

In [19]:
# Mostrar el valor máximo de la columna "day"
max_day = df_clean['day'].max()
print("El valor máximo de 'day' es:", max_day)

# Mostrar el valor mínimo de la columna "day"
min_day = df_clean['day'].min()
print("El valor mínimo de 'day' es:", min_day)


El valor máximo de 'day' es: 31
El valor mínimo de 'day' es: 1


In [20]:
# Seleccionar las filas donde "previous" es mayor que 70
previous_above_70_indices = df_clean.loc[df_clean['previous'] > 70].index

# Eliminar las filas donde "previous" es mayor que 70
df_clean = df_clean.drop(previous_above_70_indices)

# Verificar la forma del DataFrame después de eliminar las filas con "previous" > 70
print("Dimensiones del DataFrame después de eliminar filas con 'previous' > 70:", df_clean.shape)


Dimensiones del DataFrame después de eliminar filas con 'previous' > 70: (45192, 17)


##4.5 Errores tipográficos en variables categóricas

En una variable categórica pueden aparecer sub-niveles como "unknown" y "UNK" que para nosotros son equivalentes pero que para nuestro programa parecerían diferentes.

Se deben unificar estos sub-niveles

### Comprobamos si hay errores tipograficos (valores como Yes, yes, YES), que representan la misma respuesta, pero escrito con mayúsculas y minúsculas

In [21]:
# Obtener las columnas categóricas del DataFrame
columnas_categoricas = df.select_dtypes(include=['object']).columns

# Iterar sobre las columnas categóricas para imprimir las categorías únicas
for columna in columnas_categoricas:
    print("Categorías únicas en la columna '{}':".format(columna))
    print(df[columna].unique())
    print()  # Agregar una línea en blanco para separar las categorías de cada columna


Categorías únicas en la columna 'job':
['management' 'technician' 'entrepreneur' 'blue-collar' 'unknown'
 'Management' 'retired' 'admin.' 'services' 'self-employed' 'MANAGEMENT'
 'Self-employed' 'unemployed' 'housemaid' 'student' 'Services' 'Retired'
 'administrative' nan]

Categorías únicas en la columna 'marital':
['married' 'single' 'div.' 'divorced' 'DIVORCED' 'Single' nan]

Categorías únicas en la columna 'education':
['tertiary' 'secondary' 'unknown' 'primary' 'SECONDARY' 'Secondary'
 'Primary' 'sec.' 'Tertiary' 'UNK' nan]

Categorías únicas en la columna 'default':
['no' 'yes']

Categorías únicas en la columna 'housing':
['yes' 'no']

Categorías únicas en la columna 'loan':
['no' 'yes' 'No' 'YES' 'Yes' 'NO']

Categorías únicas en la columna 'contact':
['unknown' 'cellular' 'telephone' 'phone' 'mobile']

Categorías únicas en la columna 'month':
['may' 'nov' 'jun' 'jul' 'aug' 'oct' 'dec' 'jan' 'feb' 'mar' 'apr' 'sep']

Categorías únicas en la columna 'poutcome':
['unknown' 'UNK' '

### Estandarizamos los valores a minúsculas

In [22]:
# Crear un diccionario de mapeo para las categorías que se deben reemplazar
mapeo_job = {
    'Management': 'management',
    'MANAGEMENT': 'management',
    'Services': 'services',
    'Retired': 'retired',
    'Self-employed': 'self-employed',
    'admin.': 'administrative'
}

# Aplicar el mapeo para reemplazar las categorías en la columna 'job'
df['job'] = df['job'].replace(mapeo_job)

# Verificar nuevamente las categorías únicas en la columna 'job'
print("Categorías únicas en la columna 'job' después del reemplazo:")
print(df['job'].unique())


Categorías únicas en la columna 'job' después del reemplazo:
['management' 'technician' 'entrepreneur' 'blue-collar' 'unknown'
 'retired' 'administrative' 'services' 'self-employed' 'unemployed'
 'housemaid' 'student' nan]


In [23]:
# Crear un diccionario de mapeo para las categorías que se deben reemplazar en cada columna
mapeo_marital = {'div.': 'divorced', 'DIVORCED': 'divorced', 'Single': 'single'}
mapeo_education = {'sec.': 'secondary', 'SECONDARY': 'secondary', 'Primary': 'primary', 'UNK': 'unknown'}
mapeo_loan = {'No': 'no', 'YES': 'yes'}
mapeo_poutcome = {'UNK': 'unknown', 'Success': 'success'}

# Aplicar el mapeo para reemplazar las categorías en cada columna
df['marital'] = df['marital'].replace(mapeo_marital)
df['education'] = df['education'].replace(mapeo_education)
df['loan'] = df['loan'].replace(mapeo_loan)
df['poutcome'] = df['poutcome'].replace(mapeo_poutcome)

# Verificar nuevamente las categorías únicas en cada columna
print("Categorías únicas en la columna 'marital' después del reemplazo:")
print(df['marital'].unique())
print()

print("Categorías únicas en la columna 'education' después del reemplazo:")
print(df['education'].unique())
print()

print("Categorías únicas en la columna 'loan' después del reemplazo:")
print(df['loan'].unique())
print()

print("Categorías únicas en la columna 'contact' después del reemplazo:")
print(df['contact'].unique())
print()

print("Categorías únicas en la columna 'poutcome' después del reemplazo:")
print(df['poutcome'].unique())


Categorías únicas en la columna 'marital' después del reemplazo:
['married' 'single' 'divorced' nan]

Categorías únicas en la columna 'education' después del reemplazo:
['tertiary' 'secondary' 'unknown' 'primary' 'Secondary' 'Tertiary' nan]

Categorías únicas en la columna 'loan' después del reemplazo:
['no' 'yes' 'Yes' 'NO']

Categorías únicas en la columna 'contact' después del reemplazo:
['unknown' 'cellular' 'telephone' 'phone' 'mobile']

Categorías únicas en la columna 'poutcome' después del reemplazo:
['unknown' 'failure' 'other' 'success']


In [24]:
# Crear un diccionario de mapeo para las categorías que se deben reemplazar en cada columna
mapeo_education = {'Secondary': 'secondary', 'Tertiary': 'tertiary'}
mapeo_loan = {'Yes': 'yes', 'NO': 'no'}
mapeo_contact = {'mobile': 'cellular', 'phone': 'telephone'}

# Aplicar el mapeo para reemplazar las categorías en cada columna
df['education'] = df['education'].replace(mapeo_education)
df['loan'] = df['loan'].replace(mapeo_loan)
df['contact'] = df['contact'].replace(mapeo_contact)

# Verificar nuevamente las categorías únicas en cada columna
print("Categorías únicas en la columna 'education' después del reemplazo:")
print(df['education'].unique())
print()

print("Categorías únicas en la columna 'loan' después del reemplazo:")
print(df['loan'].unique())
print()

print("Categorías únicas en la columna 'contact' después del reemplazo:")
print(df['contact'].unique())
print()

print("Categorías únicas en la columna 'poutcome' después del reemplazo:")
print(df['poutcome'].unique())


Categorías únicas en la columna 'education' después del reemplazo:
['tertiary' 'secondary' 'unknown' 'primary' nan]

Categorías únicas en la columna 'loan' después del reemplazo:
['no' 'yes']

Categorías únicas en la columna 'contact' después del reemplazo:
['unknown' 'cellular' 'telephone']

Categorías únicas en la columna 'poutcome' después del reemplazo:
['unknown' 'failure' 'other' 'success']


In [25]:
# Filtrar las filas con valores nulos en la columna 'education'
filas_con_nan_education = df[df['education'].isnull()]

# Imprimir las filas que tienen valores nulos en 'education'
print("Filas con valores nulos en 'education':")
print(filas_con_nan_education)


Filas con valores nulos en 'education':
       age          job   marital education default  balance housing loan  \
37658   43  blue-collar  divorced       NaN      no    688.0     yes   no   

        contact  day month  duration  campaign  pdays  previous poutcome   y  
37658  cellular   14   may     149.0         3   -1.0         0  unknown  no  


In [26]:
# Seleccionar las filas con valores nulos en la columna 'education'
education_null_indices = df_clean[df_clean['education'].isnull()].index

# Eliminar las filas con valores nulos en la columna 'education'
df_clean = df_clean.drop(education_null_indices)

# Mostrar el número de filas después de eliminar las filas con valores nulos en 'education'
print("Número de filas después de eliminar filas con valores nulos en 'education':", len(df_clean))


Número de filas después de eliminar filas con valores nulos en 'education': 45192


## Conclusiones

Después de limpiar el dataset, pasamos de 45215 filas a 45192, tras eliminar registros repetidos, outliers, valores negativos en variables donde claramente son un error (como en el caso de duration), además de corregir errores tipográficos en las variables categóricas.